## SSH key pair generation

we create a SSH key pair that will be used to connect to the deployed node.

In [ ]:
%x(ssh-keygen -b 2048 -t rsa -f /tmp/sshkey -q -N "")

## Job and deployment definition

In [ ]:
require 'xp5k'
myxp = XP5K::XP.new
myxp.define_job({
  :resources => ["{cluster='hercule'}/nodes=4,{cluster='sagittaire'}/nodes=4, walltime=1:00:00"],
    :site      => 'lyon',
    :types     => ["deploy"],
    :name      => "iruby notebook" ,
    :roles     => [XP5K::Role.new({:name => 'mynode', :size => 8})],
    :command   => "sleep 86400"
    })
myxp.define_deployment({
    :site           => 'lyon',
    :environment    => "jessie-x64-nfs",
    :roles          => %w{ mynode },
    :key            => File.read("/tmp/sshkey.pub")
})

## Job and deployment submission

In [ ]:
myxp.submit
myxp.wait_for_jobs
myxp.deploy

## Install fping on all the nodes

In [ ]:
require 'xp5k/rake'
h = on roles('mynode'), {:user => 'root', :ssh => {:keys => ['/tmp/sshkey']}} do 
  cmd = ['apt-get update -qq && apt-get install -qy fping']
end

## Each node pings all nodes

In [ ]:
servers = roles('mynode')
h = on roles('mynode'), {:user => 'root', :ssh => {:keys => ['/tmp/sshkey']}} do 
  cmd = "cat > /tmp/hosts << EOF
#{servers.join("\n")}
EOF"
end

In [ ]:
n = 100
h = on roles('mynode'), {:user => 'root', :ssh => {:keys => ['/tmp/sshkey']}} do 
  cmd = "fping -C #{n} -q -s -e -f /tmp/hosts"
end

In [ ]:
hh = []
h.each do |k,v|
    hh.concat(h[k]
      .split("\n")[0..servers.size - 1]
        .map{|x| x.split(" ")}
        .map{|a| [k, a[0], a[2..-1].map{|l| l.to_f/n}.sum]})
end
hh.size

In [ ]:
node_to_key = h.keys.sort.each_with_index.inject([]){ |h,i| h<<i}.to_h
key_to_node = node_to_key.map{|k,v| [v,k]}.to_h

In [ ]:
x=hh.map{|x| node_to_key[x[0]]}
y=hh.map{|x| node_to_key[x[1]]}
fill=hh.map{|x|x[2]}

# Draw latency (ms)

In [ ]:
require 'nyaplot'
plot5 = Nyaplot::Plot.new
hm = plot5.add(:heatmap, x, y, fill)
hm.stroke_color("#fff")
hm.stroke_width("0")
hm.width(1)
hm.height(1)
plot5.legend(true)
plot5.x_label("hosts")
plot5.y_label("hosts")
plot5.show



In [ ]:
myxp.clean